In [3]:
import torch
from datasets import load_dataset

In [4]:
!pip install peft
!pip install ipywidgets

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
squad = load_dataset("squad", split="train[:5000]").train_test_split(test_size=0.2)

In [7]:
from transformers import AutoTokenizer

my_model = "sharmadhruv/qa_by_bird_lora_weights"
checkpoint = "google/bigbird-pegasus-large-pubmed"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [8]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    end_questions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
            
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
        end_questions.append(len(questions[i]))

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs['end_questions'] = end_questions
    return inputs

In [9]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [11]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

config = PromptTuningConfig(
    task_type=TaskType.QUESTION_ANS,             # This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.RANDOM,  # The added virtual tokens are initializad with random numbers
    num_virtual_tokens=10,                       # Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=checkpoint,           # The pre-trained model.
)

model = AutoModelForQuestionAnswering.from_pretrained(checkpoint, low_cpu_mem_usage=True)
peft_model = get_peft_model(model, config)

Some weights of BigBirdPegasusForQuestionAnswering were not initialized from the model checkpoint at google/bigbird-pegasus-large-pubmed and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
print(peft_model.print_trainable_parameters())

trainable params: 12,290 || all params: 576,906,244 || trainable%: 0.0021
None


In [13]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
    max_new_tokens=50, do_sample=True, top_k=50, eos_token_id=model.config.eos_token_id
)
generation_config.save_pretrained(my_model, push_to_hub=True)

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=my_model,
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    report_to = "none",
    save_strategy='no'
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [15]:
trainer.train()
trainer.push_to_hub()

ValueError: If no `decoder_input_ids` or `decoder_inputs_embeds` are passed, `input_ids` cannot be `None`. Please pass either `input_ids` or `decoder_input_ids` or `decoder_inputs_embeds`.